Test with Llama 3.2 3B using unsloth and function calling

Llama 3.2 prompt format
https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/

In [1]:
folder_path = ''

In [2]:
from transformers import pipeline
import json
import wandb
with open(folder_path+'../tokens.json', 'r') as file_handler:
    tokens = json.load(file_handler)

# Defined in the secrets tab in Google Colab
hf_token = tokens['hf_token']
wb_token = tokens['wandb_token']
wandb.login(key=wb_token)

/home/penhfel/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: felipe-fonseca. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/penhfel/.netrc


True

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: NVIDIA GeForce RTX 4080. Max memory: 15.992 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): L

In [4]:
system_prompt_with_tools = """You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,also point it out. You should only return the function call in tools call sections.
If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke.[
    {
        "name": "get_user_info",
        "description": "Retrieve details for a specific user by their unique identifier. Note that the provided function is in Python 3 syntax.",
        "parameters": {
            "type": "dict",
            "required": [
                "user_id"
            ],
            "properties": {
                "user_id": {
                "type": "integer",
                "description": "The unique identifier of the user. It is used to fetch the specific user details from the database."
            },
            "special": {
                "type": "string",
                "description": "Any special information or parameters that need to be considered while fetching user details.",
                "default": "none"
                }
            }
        }
    }
]"""
messages = [
            {"role": "system", "content": system_prompt_with_tools},
            {
                "role": "user",
                "content": "Can you retrieve the details for the user with the ID 7890, who has black as their special request?",
            }
        ]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)

In [5]:
input_ids = tokenizer(inputs, return_tensors="pt").to("cuda")
output = model.generate(**input_ids, max_new_tokens=100)

print(tokenizer.decode(output[0], skip_special_tokens=True))

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,also point it out. You should only return the function call in tools call sections.
If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke.[
    {
        "name": "get_user_info",
        "description": "Retrieve details for a specific user by their unique identifier. Note that the provided function is in Python 3 syntax.",
        "parameters": {
            "type": 

In [6]:
custom_chat_template = """{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "26 Jul 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none  %}
    {{- "You are an expert in composing functions. You are given a question and a set of possible functions.\n" }}
    {{- "Based on the question, you will need to make one or more function/tool calls to achieve the purpose.\n" }}
    {{- "If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,also point it out. You should only return the function call in tools call sections.\n" }}
    {{- "If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)].\n" }}
    {{- 'You SHOULD NOT include any other text in the response.\n' }}
    {{- 'You SHOULD ONLY choose one of the given functions.\n' }}
    {{- "Here is a list of functions in JSON format that you can invoke.[\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- ",\n" }}
    {%- endfor %}
    {{- "]\n" }}
{%- endif %}
{{- system_message }}
{{- "<|eot_id|>" }}

{%- for message in messages %}
    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}
        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' }}
    {%- elif 'tool_calls' in message %}
        {%- if not message.tool_calls|length == 1 %}
            {{- raise_exception("This model only supports single tool-calls at once!") }}
        {%- endif %}
        {%- set tool_call = message.tool_calls[0].function %}
        {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
        {{- '{"name": "' + tool_call.name + '", ' }}
        {{- '"parameters": ' }}
        {{- tool_call.arguments | tojson }}
        {{- "}" }}
        {{- "<|eot_id|>" }}
    {%- elif message.role == "tool" or message.role == "ipython" %}
        {{- "<|start_header_id|>ipython<|end_header_id|>\n\n" }}
        {%- if message.content is mapping or message.content is iterable %}
            {{- message.content | tojson }}
        {%- else %}
            {{- message.content }}
        {%- endif %}
        {{- "<|eot_id|>" }}
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
{%- endif %}
"""

In [7]:
tokenizer.chat_template =  custom_chat_template

In [8]:
def chatbot_cartao_de_credito(pergunta_sobre_cartao: str):
    """
    Faça perguntas para um chatbot especializado no assunto de cartão de crédito

    Args:
        pergunta_sobre_cartao: Pergunta do usuário que leva em consideração o histórico da conversa
    """
    return 'I know about credit cards'

def chatbot_sobre_transferencia(pergunta_sobre_transf: str):
    """
    Faça perguntas para um chatbot especializado no assunto de transferencias bancárias

    Args:
        pergunta_sobre_transf: Pergunta do usuário que leva em consideração o histórico da conversa
    """
    return '10'

In [9]:
messages = [
    {
        "role": "user",
        "content": "Como eu posso pedir um cartão de crédito?",
    },
    {
        "role": "assistant",
        "content": "Eu tenho uma lista de cartões para te oferecer. Qual desses cartões você quer solicitar? Black, Visa ou Infinite?",
    },
    {
        "role": "user",
        "content": "o visa",
    }
]

In [10]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False, tools=[chatbot_cartao_de_credito,chatbot_sobre_transferencia])

In [11]:
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
output = model.generate(**input_ids, max_new_tokens=100)

print(tokenizer.decode(output[0][len(input_ids.input_ids[0]):], skip_special_tokens=True))

assistant

[chatbot_cartao_de_credito(pergunta_sobre_cartao='Como eu posso pedir um cartão de crédito?')]
